# Elastic Cache - Redis Enterprise Comparison

A notebook to create a memorybound EC vs RE comparison.

Using single DB RE deployments compaired side by side to their equivalent EC deployments.

In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# Get top level path
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)
toppath = os.getcwd()

# EC Raw Data
### Transform Raw EC data into useable datatable
All EC node types and associated meta data, all EC node type Reserved prices

Combine into clean datasheet

In [ ]:
path = toppath+"/data/EC-Supported-Node-Types.csv"
df_ec_nodes = pd.read_csv(path, sep=",")
df_ec_nodes.tail(2)

# remove Data Tiering Nodes for this analysis
df_ec_nodes = df_ec_nodes[(df_ec_nodes['Node Version']!='Memory Optimized Cache Nodes with data tiering - Current Generation')]
df_ec_nodes.tail(1)

# Reserved prices, 1 year and 3 year
path = toppath+"/data/Reserved-EC-Prices.csv"
df_ec_prices = pd.read_csv(path, sep=",")
df_ec_prices.head(2)

df_ec_prices_1year = df_ec_prices[(df_ec_prices['Reservation Years']=='1 year')]
df_ec_prices_1year = df_ec_prices_1year.add_prefix('1 Year ')
df_ec_prices_1year.tail(1)

df_ec_prices_3year = df_ec_prices[(df_ec_prices['Reservation Years']=='3 year')]
df_ec_prices_3year = df_ec_prices_3year.add_prefix('3 Year ')
df_ec_prices_3year.tail(1)


### Merge data tables
###(note, not all EC instances have reserved prices, so there are some nan values)
df_ec_nodes_prices = df_ec_nodes.merge(df_ec_prices_1year, how='left', left_on='Cache Node Type', right_on='1 Year Node type')
df_ec_nodes_prices = df_ec_nodes_prices.merge(df_ec_prices_3year, how='left', left_on='Cache Node Type', right_on='3 Year Node type')

#### Clean datatable to usable format
df_ec_nodes_prices.rename(columns = {'Cache Node Type':'EC Cache Node Type',
                     'EC reserved memory percent (25%)':'Max Memory per Node (25% EC Reduction)',
                     'vCPU':'EC vCPU',
                     'Memory (GiB)':'EC RAM (GB)',
                    'Network Performance':'EC Network (GiB)',
                    'Price Per Hour (On-Demand) (US East (Ohio))':'EC On-Demand Cost/Hour/Node',
                    '1 Year RI effective hourly rate**':'EC 1 Year RI effective hourly rate/Node',
                    '3 Year RI effective hourly rate**':'EC 3 Year RI effective hourly rate/Node'},
          inplace = True)

# remove unnessesary columns
df_ec_nodes_prices = df_ec_nodes_prices[['EC Cache Node Type',
                    'Max Memory per Node (25% EC Reduction)',
                    'EC vCPU',
                    'EC RAM (GB)',
                   'EC Network (GiB)',
                    'EC On-Demand Cost/Hour/Node',
                   'EC 1 Year RI effective hourly rate/Node',
                   'EC 3 Year RI effective hourly rate/Node']]


# Create EC node deployment datatable

EC has 46 nodes but multiple deployment types:
* 1 master no HA, 
* 1 master with HA (1-5 extra nodes)
* Cluster mode:
** multiple masters (up to 500 total nodes (including replicas) 

so each individual node can be deployed from a single master with no HA to a 500 node cluster with no HA, to a 250 master node, 250 replica node set up, etc.

#### add additional columns
* HA (Yes/No)
* Cluster Mode (Yes/No)
* Master Nodes Count
* HA Nodes Count
* Total Nodes (Master + HA)
     * calc: (master nodes count * (HA nodes count + 1))
* Total Dataset Size Possible (GB) (Master Nodes)
* Total Memory Limit Possible (GB) (including replication)

In [ ]:
path = toppath+"/data/EC-Deployment-Options.csv"
df_ec_deployment = pd.read_csv(path, sep=",")
df_ec_deployment.head()

ec_node_type_list = df_ec_nodes_prices['EC Cache Node Type'].tolist()
ec_node_type_list[0:3]

# create new df, duplicate each ec deployment node options to each individual node type.
# there are 1223 node deployment options per node type, there are 43 node types. 
# so a total of 56303 deployment options in EC

df_ec_node_deployment_options = pd.DataFrame()
for i in ec_node_type_list:
    # each node type
    print(i)
    # add each node type to the deployment options
    df_ec_deployment_1 = df_ec_deployment.copy()
    df_ec_deployment_1['EC Cache Node Type'] = i
    df_ec_deployment_1.reset_index(inplace=True)
    df_ec_deployment_1 = df_ec_deployment_1.rename(columns = {'index':'id'})
    # append each to a new df
    df_ec_node_deployment_options = pd.concat([df_ec_node_deployment_options, df_ec_deployment_1])
    print(df_ec_node_deployment_options.shape)
    
    
df_ec_node_deployment_options['EC Cache Node Type (id)'] = df_ec_node_deployment_options['EC Cache Node Type'] + " " + df_ec_node_deployment_options['id'].astype(str)
df_ec_node_deployment_options.tail()


#### Combine the EC node deployment options to the EC node metadata
df_ec_node_dply = df_ec_nodes_prices.merge(df_ec_node_deployment_options, how='left', left_on='EC Cache Node Type', right_on='EC Cache Node Type')

### Perform calculations on merged data
####Now that we have merged the EC node data to the EC deployment options metadata we can calculate deployment size and cost information.
df_ec_node_dply["Total Dataset Size Possible (GB) (Master Nodes)"] = df_ec_node_dply["Master Nodes Count"] * df_ec_node_dply["Max Memory per Node (25% EC Reduction)"]

df_ec_node_dply["Total Memory Limit Possible (GB) (Including Replication)"] = df_ec_node_dply["Total Nodes (Master + Replica)"] * df_ec_node_dply["Max Memory per Node (25% EC Reduction)"]

df_ec_node_dply["Total On Demand Cost/Hour (All Nodes)"] = df_ec_node_dply["EC On-Demand Cost/Hour/Node"] * df_ec_node_dply["Total Nodes (Master + Replica)"]

df_ec_node_dply["Total On Demand Cost/Month (All Nodes)"] = df_ec_node_dply["Total On Demand Cost/Hour (All Nodes)"] * 730

df_ec_node_dply["Total EC 1 RI Cost/Hour (All Nodes)"] = df_ec_node_dply["EC 1 Year RI effective hourly rate/Node"] * df_ec_node_dply["Total Nodes (Master + Replica)"]

df_ec_node_dply["Total EC 1 RI Cost/Month (All Nodes)"] = df_ec_node_dply["Total EC 1 RI Cost/Hour (All Nodes)"] * 730

df_ec_node_dply["Total EC 3 RI Cost/Hour (All Nodes)"] = df_ec_node_dply["EC 3 Year RI effective hourly rate/Node"] * df_ec_node_dply["Total Nodes (Master + Replica)"]

df_ec_node_dply["Total EC 3 RI Cost/Month (All Nodes)"] = df_ec_node_dply["Total EC 3 RI Cost/Hour (All Nodes)"] * 730


df_ec_node_dply.tail()


path = toppath+"/data/df_ec_node_deployment_options.csv"
df_ec_node_dply.to_csv(path, index=False)

# Redis Enterprise Flexible Plan

Use RE Flex Plan Datasheets for growing Db datasets.

Large and High Throughput Shard types with and without HA.

RE data set size Maxes out at 6250 for large shards and 625 for High throughput.

Why? Because that is 500 shards. 

To my understanding going beyond a 500 shard RE database is tricky territory.

In [ ]:
path = toppath+"/data/RE Flex Deployment.xlsx"
df_re_large = pd.read_excel(path, sheet_name='Flex Large')
df_re_large.tail(2)

path = toppath+"/data/RE Flex Deployment.xlsx"
df_re_ht = pd.read_excel(path, sheet_name='Flex High Throughput')
df_re_ht.tail(2)

# Concat Large shard deployments and High throughput shard deployment options together
df_re = pd.concat([df_re_large,df_re_ht])
print(df_re.shape)
df_re.tail(2)

# Putting it All Together (Merging EC and RE)

Merging the tables together to get an acurate picture of RE vs EC means we need to examine each EC node deployment option (56304 unique options) to each individual RE Flex option (1500 unique options).

This can be reduced though, we only need to look at HA vs HA and non-HA vs non-HA.

It can be further reduced once the initial merge is done per RE Flex option to filter the options to within a percentage of total dataset size in the plus and minus direction.
* (ie. keep only something like (-20% - +20%) RE total dataset size to EC total dataset size ). 
    * This is because we want to try and keep it apples to apples in some respect. Not compare a 25GB deployment to a 1TB deployment which doesnt make sense.

A further reduction can be done to keep master shard counts relativly similar.
* (ie. RE deployment has 10 master nodes, EC should have between 7 and 13 master nodes)

#### How do we do this:

First filter both dfs to be HA vs non HA tables.

Then:

EC-Node-DF on the left side, add a single row from the RE-df table 
* (ie. a single RE flex deployment).

Then peform calculations on datasize comparison between EC vs RE.

Filter to within some pecent range (ie. +/-20%) of total dataset size.

Then repeate the step and append to the final datatable.

In [ ]:
# RE Datatable
print(df_re.shape)
df_re.tail(2)

In [ ]:
# EC datatable
print(df_ec_node_dply.shape)
df_ec_node_dply.tail(2)

### User Inputs
#### Choose the Dataset size range you are interested in viewing
#### Choose the range of EC deployment sizes to compare too
#### Choose number of HA nodes to include


In [ ]:
############################### User inputs for RE vs EC comparison export

### pick your dataset gb range
min_RE_dataset_size_GB = 0
max_RE_dataset_size_GB = 3000


# you will want a range of total dataset GB size change.
# this will allow you to view EC deployments within a range of your RE deployment
# example: RE deployment dataset size = 100, view 15% +/- range and see all EC deployments between 75GB & 115GB
###### !!!!! (keep this relatively small, otherwise the combo sheet could be millions of rows)
min_percent_total_dataset_gb_change = -0.15
max_percent_total_dataset_gb_change = 0.15

# Same as above but with cluster size (master shard count range)
# You may want to compare similar deployments, 
### no need to look at a RE deployment of 4 master shards compared to a EC deployment of 500 masters or vice versa
### the percent differences may be more confusing here though.
### example: EC deployment 10 masters: 
##### min shard count percent change of -80% means EC of 10 should only be compared to RE deployments of at minimum 2 masters
##### max shard count percent change of 100 means, 1000% increase, 
##### so 10 EC shard deployment could be compared to an RE deployment of 1000 shards is 40 shards.

# if you do not want to limit here, just put -1 as min, and 1000 as max. then it wont filter on anything.
min_percent_total_master_shard_count_change = -0.8
max_percent_total_master_shard_count_change = 100


# min HA nodes must be >= 1, max can be up to 5
# keeping the max to 2 reduces the datasheet size. making it more usable.
# if you have over 2 HA nodes RE is most likley always cheaper anyway
min_HA_Nodes_Count = 1
max_HA_Nodes_Count = 2

# Run The Comparison:

In [ ]:
# Filter tables to Non-HA
print(df_re.shape)

df_re_noHA = df_re[(df_re['HA (Yes/No)'])=='No']
df_re_noHA = df_re_noHA[(df_re_noHA['Shard Type'])=='Large']
#df_re_noHA = df_re_noHA[(df_re_noHA['Dataset Size (GB)'])<=3000]
df_re_noHA = df_re_noHA[(df_re_noHA['Dataset Size (GB)']>=min_RE_dataset_size_GB) & (df_re_noHA['Dataset Size (GB)']<=max_RE_dataset_size_GB)]
print(df_re_noHA.shape)
#df_re_noHA.tail(2)

print(df_ec_node_dply.shape)
df_ec_node_dply_noHA = df_ec_node_dply[(df_ec_node_dply['HA Nodes Count']==0)]
#df_ec_node_dply_noHA = df_ec_node_dply_noHA[(df_ec_node_dply_noHA['Total Dataset Size Possible (GB) (Master Nodes)']<=3500)]
print(df_ec_node_dply_noHA.shape)
#df_ec_node_dply_noHA.tail(2)

### NO HA Table
df_re_noHA.reset_index()
df_re_noHA = df_re_noHA.add_prefix('RE-')
#df_re_noHA

df_ec_node_dply_noHA1 = df_ec_node_dply_noHA
print(df_ec_node_dply_noHA1.shape)
df_ec_node_dply_noHA1.head(1)


# iterate through rows:
df_all_NoHA = pd.DataFrame()
for index, row in df_re_noHA.iterrows():
    #print(row)
    #print('**********')
    
    count = -1
    #df_all = pd.DataFrame()
    #print(df_ec_node_dply_noHA1.shape)
    df_ec_node_dply_noHA = df_ec_node_dply_noHA1.copy()
    for i in row:
        count +=1
        #print(row.index[count])
        #print(i)
        df_ec_node_dply_noHA[row.index[count]] = i
    #print(df_ec_node_dply_noHA.shape)    
    df_ec_node_dply_noHA["Total Deployment Size Delta (GB) (RE - EC)"] = df_ec_node_dply_noHA["RE-Total Dataset Size (Including Replication) (GB)"] - df_ec_node_dply_noHA['Total Memory Limit Possible (GB) (Including Replication)']

    df_ec_node_dply_noHA["% Deployment size Change (RE vs EC)"] = (df_ec_node_dply_noHA['RE-Total Dataset Size (Including Replication) (GB)'] - df_ec_node_dply_noHA['Total Memory Limit Possible (GB) (Including Replication)'])/abs(df_ec_node_dply_noHA['Total Memory Limit Possible (GB) (Including Replication)'])

    df_ec_node_dply_noHA["Total Master shard Size Delta (GB) (RE - EC)"] = df_ec_node_dply_noHA['RE-Dataset Size (GB)'] - df_ec_node_dply_noHA['Total Dataset Size Possible (GB) (Master Nodes)']

    df_ec_node_dply_noHA["% Total Master Shard Size Change (RE vs EC)"] = (df_ec_node_dply_noHA['RE-Dataset Size (GB)'] - df_ec_node_dply_noHA['Total Dataset Size Possible (GB) (Master Nodes)'])/abs(df_ec_node_dply_noHA['Total Dataset Size Possible (GB) (Master Nodes)'])

    df_ec_node_dply_noHA["ON DEMAND Price Delta (RE - EC)"] = df_ec_node_dply_noHA['RE-Cost/Month'] - df_ec_node_dply_noHA['Total On Demand Cost/Month (All Nodes)']

    df_ec_node_dply_noHA["ON DEMAND % Change (RE vs EC)"] = (df_ec_node_dply_noHA['RE-Cost/Month'] - df_ec_node_dply_noHA['Total On Demand Cost/Month (All Nodes)'])/abs(df_ec_node_dply_noHA['Total On Demand Cost/Month (All Nodes)'])

    df_ec_node_dply_noHA["Total Shard Count Delta (RE - EC)"] = df_ec_node_dply_noHA['RE-Total Shard Count'] - df_ec_node_dply_noHA['Total Nodes (Master + Replica)']

    df_ec_node_dply_noHA["% Total Shard Count Change (RE - EC)"] = (df_ec_node_dply_noHA['RE-Total Shard Count'] - df_ec_node_dply_noHA['Total Nodes (Master + Replica)'])/abs(df_ec_node_dply_noHA['Total Nodes (Master + Replica)'])

    df_ec_node_dply_noHA["Total Master Shard Count Delta (RE - EC)"] = df_ec_node_dply_noHA['RE-Master Shard Count'] - df_ec_node_dply_noHA['Master Nodes Count']

    df_ec_node_dply_noHA["% Total Master Shard Count Change (RE - EC)"] = (df_ec_node_dply_noHA['RE-Master Shard Count'] - df_ec_node_dply_noHA['Master Nodes Count'])/abs(df_ec_node_dply_noHA['Master Nodes Count'])    
    
    #print(df_ec_node_dply_noHA.shape)
    # Filter
    df_ec_node_dply_noHA = df_ec_node_dply_noHA[(df_ec_node_dply_noHA['% Total Master Shard Size Change (RE vs EC)']>=min_percent_total_dataset_gb_change) & (df_ec_node_dply_noHA['% Total Master Shard Size Change (RE vs EC)']<=max_percent_total_dataset_gb_change)]
    df_ec_node_dply_noHA = df_ec_node_dply_noHA[(df_ec_node_dply_noHA['% Total Master Shard Count Change (RE - EC)']>=min_percent_total_master_shard_count_change) & (df_ec_node_dply_noHA['% Total Master Shard Count Change (RE - EC)']<=max_percent_total_master_shard_count_change)]

    #print(df_ec_node_dply_noHA.shape)
    print("***")
    #df_all_NoHA = df_all_NoHA.append(df_ec_node_dply_noHA)
    df_all_NoHA = pd.concat([df_all_NoHA, df_ec_node_dply_noHA])
    print(df_all_NoHA.shape)
    
print(df_ec_node_dply_noHA1.shape)
print(df_all_NoHA.shape)
#df_all_NoHA.head(2)


########## HA Nodes Comparison

# Filter tables to Non-HA
print(df_re.shape)

df_re_HA = df_re[(df_re['HA (Yes/No)'])=='Yes']
df_re_HA = df_re_HA[(df_re_HA['Shard Type'])=='Large']
df_re_HA = df_re_HA[(df_re_HA['Dataset Size (GB)']>=min_RE_dataset_size_GB) & (df_re_HA['Dataset Size (GB)']<=max_RE_dataset_size_GB)]
print(df_re_HA.shape)
df_re_HA.head(2)

print(df_ec_node_dply.shape)
df_ec_node_dply_HA = df_ec_node_dply[(df_ec_node_dply['HA Nodes Count']>=min_HA_Nodes_Count) & (df_ec_node_dply['HA Nodes Count']<=max_HA_Nodes_Count)]

print(df_ec_node_dply_HA.shape)
df_ec_node_dply_HA.tail(2)

df_re_HA.reset_index()
df_re_HA = df_re_HA.add_prefix('RE-')
df_re_HA.head(1)

df_ec_node_dply_HA1 = df_ec_node_dply_HA
print(df_ec_node_dply_HA1.shape)
#df_ec_node_dply_HA1.head(1)


# iterate through rows:
df_all_HA = pd.DataFrame()
for index, row in df_re_HA.iterrows():
    #print(row)
    #print('**********')
    
    count = -1
    #df_all = pd.DataFrame()
    #print(df_ec_node_dply_HA1.shape)
    df_ec_node_dply_HA = df_ec_node_dply_HA1.copy()
    for i in row:
        count +=1
        #print(row.index[count])
        #print(i)
        df_ec_node_dply_HA[row.index[count]] = i
    #print(df_ec_node_dply_HA.shape)    
    df_ec_node_dply_HA["Total Deployment Size Delta (GB) (RE - EC)"] = df_ec_node_dply_HA["RE-Total Dataset Size (Including Replication) (GB)"] - df_ec_node_dply_HA['Total Memory Limit Possible (GB) (Including Replication)']

    df_ec_node_dply_HA["% Deployment size Change (RE vs EC)"] = (df_ec_node_dply_HA['RE-Total Dataset Size (Including Replication) (GB)'] - df_ec_node_dply_HA['Total Memory Limit Possible (GB) (Including Replication)'])/abs(df_ec_node_dply_HA['Total Memory Limit Possible (GB) (Including Replication)'])

    df_ec_node_dply_HA["Total Master shard Size Delta (GB) (RE - EC)"] = df_ec_node_dply_HA['RE-Dataset Size (GB)'] - df_ec_node_dply_HA['Total Dataset Size Possible (GB) (Master Nodes)']

    df_ec_node_dply_HA["% Total Master Shard Size Change (RE vs EC)"] = (df_ec_node_dply_HA['RE-Dataset Size (GB)'] - df_ec_node_dply_HA['Total Dataset Size Possible (GB) (Master Nodes)'])/abs(df_ec_node_dply_HA['Total Dataset Size Possible (GB) (Master Nodes)'])

    df_ec_node_dply_HA["ON DEMAND Price Delta (RE - EC)"] = df_ec_node_dply_HA['RE-Cost/Month'] - df_ec_node_dply_HA['Total On Demand Cost/Month (All Nodes)']

    df_ec_node_dply_HA["ON DEMAND % Change (RE vs EC)"] = (df_ec_node_dply_HA['RE-Cost/Month'] - df_ec_node_dply_HA['Total On Demand Cost/Month (All Nodes)'])/abs(df_ec_node_dply_HA['Total On Demand Cost/Month (All Nodes)'])

    df_ec_node_dply_HA["Total Shard Count Delta (RE - EC)"] = df_ec_node_dply_HA['RE-Total Shard Count'] - df_ec_node_dply_HA['Total Nodes (Master + Replica)']

    df_ec_node_dply_HA["% Total Shard Count Change (RE - EC)"] = (df_ec_node_dply_HA['RE-Total Shard Count'] - df_ec_node_dply_HA['Total Nodes (Master + Replica)'])/abs(df_ec_node_dply_HA['Total Nodes (Master + Replica)'])

    df_ec_node_dply_HA["Total Master Shard Count Delta (RE - EC)"] = df_ec_node_dply_HA['RE-Master Shard Count'] - df_ec_node_dply_HA['Master Nodes Count']

    df_ec_node_dply_HA["% Total Master Shard Count Change (RE - EC)"] = (df_ec_node_dply_HA['RE-Master Shard Count'] - df_ec_node_dply_HA['Master Nodes Count'])/abs(df_ec_node_dply_HA['Master Nodes Count'])
    #print(df_ec_node_dply_HA.shape)
    # Filter
    df_ec_node_dply_HA = df_ec_node_dply_HA[(df_ec_node_dply_HA['% Total Master Shard Size Change (RE vs EC)']>=min_percent_total_dataset_gb_change) & (df_ec_node_dply_HA['% Total Master Shard Size Change (RE vs EC)']<=max_percent_total_dataset_gb_change)]
    df_ec_node_dply_HA = df_ec_node_dply_HA[(df_ec_node_dply_HA['% Total Master Shard Count Change (RE - EC)']>=min_percent_total_master_shard_count_change) & (df_ec_node_dply_HA['% Total Master Shard Count Change (RE - EC)']<=max_percent_total_master_shard_count_change)]
    
    #print(df_ec_node_dply_HA.shape)
    print("***")
    #df_all_HA = df_all_HA.append(df_ec_node_dply_HA)
    df_all_HA = pd.concat([df_all_HA,df_ec_node_dply_HA])
    print(df_all_HA.shape)
    
    
print(df_all_HA.shape)
df_all_HA.head(2)


######## Combine them together
#df_all_combo = df_all_NoHA.append(df_all_HA)
df_all_combo = pd.concat([df_all_NoHA,df_all_HA])
print(df_all_combo.shape)

#df_all_combo.head(1)
df_all_combo.drop(['id','EC Cache Node Type (id)',
                   'RE-Min Cluster Cost/Hour',
                   'RE-Shard Cost/Hour',
                   'EC On-Demand Cost/Hour/Node',
                   'EC 1 Year RI effective hourly rate/Node',
                   'EC 3 Year RI effective hourly rate/Node',
                  ], axis=1)
df_all_combo.head(1)

In [ ]:
path = toppath+"/data/RE-EC-comparison-export.csv"
df_all_combo.to_csv(path, index=False)

# NEXT STEP

find all EC deployments larger than RE that are also cheaper than RE

In [ ]:

df_all_combo1 = df_all_combo[(df_all_combo['ON DEMAND Price Delta (RE - EC)']>=0)]
df_all_combo1 = df_all_combo1[(df_all_combo1['Total Master shard Size Delta (GB) (RE - EC)']<=0)]
print(df_all_combo1.shape)
df_all_combo1.head(3)

In [ ]:
path = toppath+"/data/RE-EC-comparison-export-EC-Bigger-and-Cheaper.csv"
df_all_combo1.to_csv(path, index=False)